In [ ]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import urllib
from datetime import date

In [449]:
activities_id, \
activities, \
config, \
context, \
girder_connection, \
postgres_connection, \
groups, \
postgres_tables, \
users = postgres_to_girder._main()

Connected to the Girder database 🏗🍃 and authenticated.
Connected to the Postgres database 🐘


In [ ]:
postgres_to_girder._delete_users(girder_connection, ["5b1830ddad97092bdb8e99d1"])

In [315]:
[t for t in postgres_tables]

['organizations', 'acts', 'answers', 'users', 'user_acts']